In [ ]:
!pip install lightkurve==2.0.11
!pip install PyWavelets

In [ ]:
import pandas as pd
import lightkurve as lk
import numpy as np
import pywt
import pickle
import os
# import mathplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
print(lk.__version__)
is_colab = False

In [ ]:
print(pywt.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/MASTER IA (Unir)/TFM/light_curves_K_stars_filter.csv')

In [ ]:
df

In [ ]:
# parametros para la ejecucion
wavelet_family='sym5'
level = 9
save_lc = True
save_path = '/content/drive/MyDrive/MASTER IA (Unir)/TFM/waveletsG/'
kep_id = 10583180
kep_id_2 =9021075
wavelet_windows = 15000

In [ ]:
completed = os.listdir(save_path)
if 'errors.txt' in completed:
  completed.remove('errors.txt')
completed_id = []
for element in completed:
  completed_id.append(element.replace('.pickle','').replace('kic ',''))
len(completed_id)

In [ ]:
period,epoch,disp=df[df['kepid']==kep_id][['koi_period','koi_time0bk','koi_disposition']].iloc[0]
period_2,epoch_2,disp_2=df[df['kepid']==kep_id_2][['koi_period','koi_time0bk','koi_disposition']].iloc[0]
print(period, epoch,disp)

In [ ]:
class LightCurveWaveletFoldCollection():
  
    def __init__(self,light_curve,wavelets):
        self._light_curve = light_curve
        self._lc_w_collection = wavelets

    def get_detail_coefficent(self,level = None):
        if level != None:
            return self._lc_w_collection[level-1][1]
        return self._lc_w_collection[:][1]

    def get_approximation_coefficent(self,level = None):
        if level != None:
            return self._lc_w_collection[level-1][0]
        return self._lc_w_collection[:][0]
    
    def get_wavelets(self):
        return self._lc_w_collection

    def plot(self):
        """
        imprime el detalle de las wavelets sobre la curva de luz.
        Parameters
        ----------
        None
        Returns
        ----------
        None.
        """
        wavelet = self._lc_w_collection
        time = self._light_curve.time.value
        data = self._light_curve.flux.value
        plt.figure(figsize=(16, 4))
        plt.plot(time,data)
        ig, axarr = plt.subplots(nrows=len(wavelet), ncols=2, figsize=(16,12))
        for i,lc_w in enumerate(wavelet):
            (data, coeff_d) = lc_w
            axarr[i, 0].plot(data, 'r')
            axarr[i, 1].plot(coeff_d, 'g')
            axarr[i, 0].set_ylabel("Level {}".format(i + 1), fontsize=14, rotation=90)
            axarr[i, 0].set_yticklabels([])
            if i == 0:
                axarr[i, 0].set_title("Approximation coefficients", fontsize=14)
                axarr[i, 1].set_title("Detail coefficients", fontsize=14)
            axarr[i, 1].set_yticklabels([])
        plt.show()

class LightCurveWaveletCollection():
    def __init__(self,id,headers,lc_par,lc_inpar):
        self.pliegue_par = lc_par
        self.pliegue_inpar = lc_inpar
        self.kepler_id = id
        self.headers = headers

    def save(self, path = ""):
        """
        Metodo que guarda la curva luz procesada en un archivo.
        Parameters
        ----------
        Path: str = ""
          ruta donde se guardara el archivo.
        Returns
        ----------
        None.
        """
        file_name = path + 'kic '+str(self.kepler_id)+'-'+self.headers['Kepler_name']+'.pickle'
        with open(file_name, "wb") as f:
            pickle.dump(self, f)

    def load(path):
        """
        Metodo que carga la curva luz procesada en un archivo.
        Parameters
        ----------
        Path: str = ""
          ruta donde se encuentra almacenado el archivo.
        Returns
        ----------
        w_loaded: LightCurveWaveletCollection
          Clase que almacena las curvas de luz par e inpar ademas de metadata del proceso de descomposicion.
        """
        with open(path, "rb") as f:
            w_loaded = pickle.load(f)
        return w_loaded

    def plot_comparative(self):
        """
        Metodo que imprime una comparativa entre la curva de luz par e impar..
        Parameters
        ----------
        None.
        Returns
        ----------
        None.
        """
        light_curve_p = self.pliegue_par._light_curve
        light_curve_i = self.pliegue_inpar._light_curve
        w_par_Collection = self.pliegue_par
        w_inpar_Collection = self.pliegue_inpar
        wavelet_p=w_par_Collection.get_wavelets()
        wavelet_i=w_inpar_Collection.get_wavelets()
        plt.figure(figsize=(26, 8))
        plt.plot(light_curve_p.time.value,light_curve_p.flux.value,c='blue',label='par')
        plt.plot(light_curve_i.time.value,light_curve_i.flux.value,c='red',label='inpar')
        
        ig, axarr = plt.subplots(nrows=len(wavelet_p), ncols=2, figsize=(26,26))
        for i,zip_curves in enumerate(zip(wavelet_p,wavelet_i)):
            (data_p, coeff_p),(data_i, coeff_i) = zip_curves
            axarr[i, 0].plot(data_p,c='blue',label='par')
            axarr[i, 0].plot(data_i, c='red',label='inpar')
            axarr[i, 1].plot(coeff_p, c='blue',label='par')
            axarr[i, 1].plot(coeff_i, c='red',label='inpar')
            axarr[i, 0].set_ylabel("Level {}".format(i + 1), fontsize=14, rotation=90)
            axarr[i, 0].set_yticklabels([])
            if i == 0:
                axarr[i, 0].set_title("Approximation coefficients", fontsize=14)
                axarr[i, 1].set_title("Detail coefficients", fontsize=14)
            axarr[i, 1].set_yticklabels([])
        plt.show()


def fold_curve(light_curve_collection, period, epoch, sigma = 20, sigma_upper = 4):
    """
    Toma la coleccion de la curvas entregadas, las pliega y devuelve una sola con todos los datos.
    
    Parameters
    ----------
    light_curve_collection: LightCurveCollection
        coleccion de curvas de luz.
    period: float
        periodo de la orbita.
    epoch: float
        tiempo de cada transcurso.
    sigma: int
        valor de desviaciones estandas
    sigma_upper: int
        valor maximo de variacion
    Returns
    ----------
    una sola curva de luz
    """
    if not is_colab:
        lc_collection = lk.LightCurveCollection([lc.remove_outliers(sigma=20, sigma_upper=4) for lc in light_curve_collection])
    
    lc_ro = lc_collection.stitch()
    
    if is_colab:
        lc_ro = lc_ro.remove_outliers(sigma=sigma, sigma_upper=sigma_upper)
    
    lc_nonans = lc_ro.remove_nans()
    lc_fold = lc_nonans.fold(period = period,epoch_time = epoch)
    lc_odd=lc_fold[lc_fold.odd_mask]
    lc_even = lc_fold[lc_fold.even_mask]
    return lc_fold,lc_odd,lc_even

def apply_wavelet(light_curve,w_family, levels,cut_border_percent=0.1):
    """
    Aplicacion de la wavelet a la curva de luz.
    
    Parameters
    ----------
    light_curve: LightCurve
        curva de luz a la que se le aplica la wavelet.
    w_family: str
        nombre de la familia de wavelet a aplicar.
    levels: int > 0
        niveles de profundida de aplicacion.
    Returns
    ----------
    lc_wavelet: [(numpy.array,numpy.array)]
        Lista con los coeficientes de aproximacion y coeficiente de detalle.
    """
    time = light_curve.time.value
    data = light_curve.flux.value
    lc_wavelet = []
    for level in range(levels):
        level_w = pywt.dwt(data, w_family)
        lc_wavelet.append(cut_border(level_w,cut_border_percent))
        #lc_wavelet.append(level_w)
        data = level_w[0]
    return LightCurveWaveletFoldCollection(light_curve,lc_wavelet)

def load_light_curve(kepler_id,mission='Kepler'):
    """
    Descarga de la curva de luz a patir de la mision y el id.
    Parameters
    ----------
    kepler_id: int
        identificador Kepler.
    mission: str = 'Kepler' 
        mision que obtuvo el regiostro.
    Returns
    ----------
    lc_collection: LightCurveCollection
        Coleccion de curvas de luz.
    """
    kic = 'KIC '+str(kepler_id)
    lc_search = lk.search_lightcurve(kic, mission=mission)
    lc_collection = lc_search.download_all()
    return lc_collection

def cut_wavelet(lightCurve,window):
    time = lightCurve.time
    data = lightCurve.flux
    flux_error = lightCurve.flux_err
    index = np.argmin(np.absolute(time))
    min_w = index - int(window/2)
    max_w = index + int(window/2)+1
    time_selected = time[min_w:max_w]
    data_selected = data[min_w:max_w]
    flux_error_selected = flux_error[min_w:max_w]
    return lk.lightcurve.FoldedLightCurve(time=time_selected,flux=data_selected,flux_err=flux_error_selected)

def cut_border(data_old,cut_percent=0.1):
    data_len_cut = int(len(data_old[0])*(cut_percent/2))
    data_new = [data[data_len_cut:(len(data)-data_len_cut)] for data in data_old ]
    return data_new
    
def process_light_curve(kepler_id,kepler_name,disp,period,epoch,w_family,levels,plot = False, plot_comparative=False,save=False, path="",wavelet_window=None,cut_border_percent=0.2):
    """
    Metodo que procesa la curva de luz para obtener sus transformadas wavelet.
    Parameters
    ----------
    kepler_id: int
        identificador Kepler.
    period: float
        valor correspondiente al periodo de la curva.   
    epoch: float
        valor correspondiente a la epoca de la curva. 
    w_family: str
        familia de wavelet a aplicar. 
    levels: int
        niveles de profundida para aplicar las wavelets
    plot: boolean = False
        imprime una grafica con los resultados de las wavelets
    plot_comparative: boolean = False
        imprime una grafica comparativa entre las wavelets de los diferentes niveles en curvas pares e inpares.
    save: boolean = False
        genera un archivo guardando la curva de luz y sus descomposiciones wavelets
    path: str
        ruta donde se guardara el archivo en caso de que el parametro save sea True
    wavelet_window: int
        tamaño de la ventana para recortar la curva de luz plegada
    cut_border_percent: float
        porcentaje del total de puntos a recortar del total existente en el nivel de la descomposicion wavelet.
    Returns
    ----------
    lc_wavelet_collection: LightCurveWaveletCollection
        Objeto que contiene la curva de luz y sus wavelets ademas de una cabecera con metadata del proceso.
    """
    # cargamos la curva de segun su Kepler_ID
    print("descargando curvas de luz...")
    lc_collection=load_light_curve(kepler_id)
    # aplicamos el pliege a las curvas de luz y las separamos en pares e inpares
    print('Aplicando pliegue y separando en pares e inpares....') 
    _,lc_inpar,lc_par = fold_curve(lc_collection,period,epoch)

    if not wavelet_window == None:
      print('Aplicando ventana ...')
      lc_inpar = cut_wavelet(lc_inpar,wavelet_window)
      lc_par = cut_wavelet(lc_par,wavelet_window)
    
    print('Aplicando wavelets...')
    # aplicamos wavelets a curvas par
    lc_w_par = apply_wavelet(lc_par,w_family,levels,cut_border_percent=cut_border_percent)
    # aplicamos wavelets a curvas inpar
    lc_w_inpar = apply_wavelet(lc_inpar,w_family,levels,cut_border_percent=cut_border_percent)
    headers = {
        "period": period,
        "epoch": epoch,
        "class": disp,
        "wavelet_family":w_family,
        "levels":levels,
        "window":wavelet_window,
        "border_cut":cut_border_percent,
        "Kepler_name":kepler_name
    }
    lc_wavelet_collection = LightCurveWaveletCollection(kepler_id,headers,lc_w_par,lc_w_inpar)
    if(plot):
        print('graficando wavelets obtenidas...')
        lc_w_par.plot()
        lc_w_inpar.plot()
    if(plot_comparative):
        print('graficando wavelets obtenidas...')
        lc_wavelet_collection.plot_comparative()
    if(save):
        print('guardando wavelets obtenidas...')
        lc_wavelet_collection.save(path)
    return lc_wavelet_collection

def process_dataset(df_koi,plot = False, plot_comparative = False,repeat_completed=True,completed=None):
    """
    Metodo que procesa el dataframe de curvas de luz para obtener sus transformadas wavelet.
    Parameters
    ----------
    df_koi: pandas.DataFrame
        Dataset con los datos de las curvas de luz a procesar.
    plot: boolean = False
        Imprime las curvas de luz y sus descomposiciones.
    plot_comparative: boolean = False:
        Imprime una comparativa entre la curva de luz par e impar con sus descomposiciones
    repeat_completed: boolean = True
        Reprocesa las curvas de luz que ya se encuentran procesadas y almacenadas. (Se ocupa para recalcular curvas de procesos anteriores incompletos)
    completed: List = None
        Lista con las curvas de luz completadas de proceso anteriores. 
    Returns
    ----------
    lc_errors: List <int>
        Lista con las curvas de luz que no se pudieron procesar.
    """
    lc_wavelets = dict()
    lc_errors = []
    for i in range (len(df_koi)):

        koi_id,koi_name,disp, period, epoch=df_koi[['kepid','kepoi_name','koi_disposition','koi_period','koi_time0bk']].iloc[i]
        percent = i*100/(len(df_koi))
        print(f'procesando curva de luz KIC {int(koi_id)}-{koi_name}[{disp}] [{percent:.0f}%]')
        if not repeat_completed and (str(koi_id)+"-"+koi_name) in completed:
          print("curva de luz procesada anteriormente")
          continue
        try:
            process_light_curve(int(koi_id),koi_name,disp,period,epoch,wavelet_family,level,plot= plot,plot_comparative=plot_comparative,save = save_lc, path = save_path,wavelet_window=wavelet_windows)
            pass
        except:
            lc_errors.append(koi_id)
            print(f'Error with KIC {koi_id}')
    f = open (save_path+'errors.txt','w')
    for lc_error in lc_errors:
        text = 'KIC '+str(lc_error)+'\n'
        f.write(text)
    f.close()
    return lc_errors
    

In [ ]:
result_window_out = process_light_curve(kep_id,"",disp,period,epoch,wavelet_family,level,plot_comparative=False)
result_window = process_light_curve(kep_id,"",disp,period,epoch,wavelet_family,level,plot_comparative=False,wavelet_window=15000)
result_window_2 = process_light_curve(kep_id_2,"",disp_2,period_2,epoch_2,wavelet_family,level,plot_comparative=False,wavelet_window=15000)

In [ ]:
result_window.plot_comparative()
result_window_2.plot_comparative()

In [ ]:
app_c_1_p = result_window.pliegue_par.get_approximation_coefficent(level=5)
app_c_2_p = result_window_2.pliegue_par.get_approximation_coefficent(level=5)

app_c_1_i = result_window.pliegue_inpar.get_approximation_coefficent(level=5)
app_c_2_i = result_window_2.pliegue_inpar.get_approximation_coefficent(level=5)

print(np.shape(app_c_1_p))
print(np.shape(app_c_2_p))
print(np.shape(app_c_1_i))
print(np.shape(app_c_2_i))
plt.plot(app_c_1_p,c='r')
plt.plot(app_c_1_i,c='b')
plt.show()
plt.plot(app_c_2_p,c='r')
plt.plot(app_c_2_i,c='b')

# Ejecucion y trabajo sobre las curvas de luz en el dataset

In [ ]:
errores = process_dataset(df,repeat_completed=False, completed=completed_id)

In [ ]:
errores

In [ ]:
all_kep_id = df["kepid"]

result = dict({"completado":0,"faltante":0})
faltantes = []
for kep_id in all_kep_id:
  if str(kep_id) in completed_id:
    result["completado"]+=1
  else:
    result["faltante"]+=1
    faltantes.append(kep_id)

print(result)
print(faltantes)
len(completed_id)

In [ ]:
path = save_path + completed[1]
print(path)
lcwC =  LightCurveWaveletCollection.load(path)
lcwC.headers
lcwC.plot_comparative()

# Visualizacion de curvas de luz

In [ ]:
selected = completed_id[2]
lc_name = 'kic '+str(selected)+'.pickle'
file_name = save_path+lc_name
lc = LightCurveWaveletCollection.load(file_name)
for i in range (0,10):
  lc_p = lc.pliegue_par.get_approximation_coefficent(level=i)
  lc_i = lc.pliegue_inpar.get_approximation_coefficent(level=i)
  print(f"[{i}]par: {len(lc_p)}, inpar: {len(lc_i)}")

In [ ]:
for lc_id in completed_id[:50]:
    lc_name = 'kic '+str(lc_id)+'.pickle'
    file_name = save_path+lc_name
    lc = LightCurveWaveletCollection.load(file_name)
    lc.plot_comparative()

In [ ]:
def plot_level_lc(level = 7 ,amount = 50):
  for lc_id in completed_id[:amount]:
    lc_name = 'kic '+str(lc_id)+'.pickle'
    file_name = save_path+lc_name
    lc = LightCurveWaveletCollection.load(file_name)
    lc_p = lc.pliegue_par.get_approximation_coefficent(level=level)
    lc_i = lc.pliegue_inpar.get_approximation_coefficent(level=level)
    plt.rcParams["figure.figsize"] = (20,3)
    plt.plot(lc_p,c="r")
    plt.plot(lc_i,c="b")
    plt.title(lc_name+":"+lc.headers['class'])
    plt.show()

def plot_level_lc_2(levels = [7] ,amount = 50):
  for lc_id in completed_id[:amount]:
    lc_name = 'kic '+str(lc_id)+'.pickle'
    file_name = save_path+lc_name
    lc = LightCurveWaveletCollection.load(file_name)
    lc_p =[ lc.pliegue_par.get_approximation_coefficent(level=level) for level in levels ]
    lc_i =[ lc.pliegue_inpar.get_approximation_coefficent(level=level) for level in levels ]
    
    ig, axarr = plt.subplots(nrows=1, ncols=len(levels), figsize=(26,26))
    for i in range(len(levels)):
      axarr[i].plot(lc_p,c='blue',label='par')
      axarr[i].plot(lc_i,c='blue',label='inpar')
      axarr[i].set_title("level:"+levels[i], fontsize=14)
    plt.title(lc_name+":"+lc.headers['class'])
    plt.show()

In [ ]:
plot_level_lc()

In [ ]:
plot_level_lc(level=6)

In [ ]:
plot_level_lc(level=8)